In [ ]:
pip install scanpy scvi-tools anndata
pip install --quiet gdown
pip install --quiet --upgrade seaborn==0.10
pip install --quiet bbknn
pip install pandas as pd
pip install pyscenic

Import packages

In [ ]:
import scanpy as sc
import anndata as ad
import os
import tarfile
import scvi
import numpy as np
import glob

#specific requirements of pyscenic
import loompy as lp
from MulticoreTSNE import MulticoreTSNE as TSNE

#data vis
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import umap

Load in data -- OLAF's VERSION. Garrett Depending on what you did you made need to make a different block.

In [ ]:
def load_data_from_tar(tar_path):
    with tarfile.open(tar_path, 'r:gz') as tar:
        tar.extractall(path=os.path.dirname(tar_path))
        base_dir = os.path.dirname(tar_path)
        work_dir = os.path.join(base_dir, "Work", os.path.basename(tar_path).replace('.tar.gz', ''))
        adata = sc.read_10x_mtx(
            work_dir, 
            var_names='gene_symbols', 
            cache=True  
        )
        return adata
    

input_directory = 'input/biostudiesarrayE-MTAB-14039'
adatas = []
for root, dirs, files in os.walk(input_directory):
    for file in files:
        if file.endswith('.tar.gz'):
            tar_path = os.path.join(root, file)
            adata = load_data_from_tar(tar_path)
            adata.obs['batch'] = file.replace('.tar.gz', '')  
            adatas.append(adata)

combined_adata = adatas[0].concatenate(adatas[1:])
combined_adata.obs_names_make_unique() #don't think this line is necessary.. precaution

Data Exploration

In [ ]:
bdata = combined_adata.copy()
bdata.obs.head()


NameError: name 'adata' is not defined

In [4]:


bdata.var.head()



NameError: name 'bdata' is not defined

In [ ]:
bdata.var_names.head()
bdata.X

In [ ]:
bdata.obs

## Quality Control

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)


adata.obs.head()



In [ ]:
sc.pl.violin(
    adata, 
    [
     'n_genes_by_counts', 
     'total_counts', 
     'pct_counts_mt'
     ],
    multi_panel=True
)

In [ ]:
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts")

### Normalization

In [ ]:
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4) 
sc.pp.log1p(adata) 
adata.raw = adata

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=4000,
    # subset=True, # to automatically subset to the 4000 genes
    layer="counts",
    flavor="seurat_v3"
)
sc.pl.highly_variable_genes(adata, log=True)

In [ ]:
adata = adata[:, adata.var.highly_variable].copy()
adata.raw.to_adata()

### Standard Workflow

In [ ]:
sc.pp.scale(adata) # z normalize the columns (genes)
sc.tl.pca(adata)


adata.obsm["X_pca"]
sc.pp.neighbors(adata) # compute nearest neighbors
sc.tl.umap(adata)
sc.pl.umap(adata, color="batch")

IF THE CLUSTERS ARE BY BATCH:: ADD THIS CODE:

In [ ]:

import scanpy.external as sce
sce.pp.bbknn(adata, batch_key="batch")
sc.tl.umap(adata)
sc.pl.umap(adata, color=["batch", "cell_types"], ncols=1)


IF IT STILL DOESN'T WORK THEN REDO THE LOAD IN AND USE SCIVI
https://ccbskillssem.github.io/assets/scvi_notebook.html

https://www.youtube.com/watch?v=EKTg9NV5hEA

## SCENIC

##### STEP 1: IDENTIFY TF with arboreto 


In [ ]:
f_tfs = "/ddn1/vol1/staging/leuven/stg_00002/lcb/cflerin/resources/allTFs_hg38.txt"

!pyscenic grn {f_loom_path_scenic} {f_tfs} -o adj.csv --num_workers 20

adjacencies = pd.read_csv("adj.tsv", index_col=False, sep='\t')
adjacencies.head()

##### STEP 2: Refine regulons through pruning 

In [ ]:

# ranking databases
f_db_glob = "/ddn1/vol1/staging/leuven/res_00001/databases/cistarget/databases/homo_sapiens/hg38/refseq_r80/mc9nr/gene_based/*feather"
f_db_names = ' '.join( glob.glob(f_db_glob) )

# motif databases
f_motif_path = "/ddn1/vol1/staging/leuven/res_00001/databases/cistarget/motif2tf/motifs-v9-nr.hgnc-m0.001-o0.0.tbl"

In [ ]:
!pyscenic ctx adj.tsv \
    {f_db_names} \
    --annotations_fname {f_motif_path} \
    --expression_mtx_fname {f_loom_path_scenic} \
    --output reg.csv \
    --mask_dropouts \
    --num_workers 20

In [ ]:
nGenesDetectedPerCell = np.sum(adata.X>0, axis=1)
percentiles = nGenesDetectedPerCell.quantile([.01, .05, .10, .50, 1])
print(percentiles)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5), dpi=150)
sns.distplot(nGenesDetectedPerCell, norm_hist=False, kde=False, bins='fd')
for i,x in enumerate(percentiles):
    fig.gca().axvline(x=x, ymin=0,ymax=1, color='red')
    ax.text(x=x, y=ax.get_ylim()[1], s=f'{int(x)} ({percentiles.index.values[i]*100}%)', color='red', rotation=30, size='x-small',rotation_mode='anchor' )
ax.set_xlabel('# of genes')
ax.set_ylabel('# of cells')
fig.tight_layout()

In [ ]:
!pyscenic aucell \
    {f_loom_path_scenic} \
    reg.csv \
    --output {f_pyscenic_output} \
    --num_workers 20

##### STEP 3: Cluster cells around regulons

In [ ]:
lf = lp.connect( f_pyscenic_output, mode='r+', validate=False )
auc_mtx = pd.DataFrame( lf.ca.RegulonsAUC, index=lf.ca.CellID)
lf.close()

In [ ]:
runUmap = umap.UMAP(n_neighbors=10, min_dist=0.4, metric='correlation').fit_transform
dr_umap = runUmap( auc_mtx )
pd.DataFrame(dr_umap, columns=['X', 'Y'], index=auc_mtx.index).to_csv( "scenic_umap.txt", sep='\t')
# tSNE
tsne = TSNE( n_jobs=20 )
dr_tsne = tsne.fit_transform( auc_mtx )
pd.DataFrame(dr_tsne, columns=['X', 'Y'], index=auc_mtx.index).to_csv( "scenic_tsne.txt", sep='\t')

#### STEP 4 Integrate the Output --- THIS LOOKS PROBLEMATIC AND VERY NOT RIGHT

In [ ]:
# scenic output
lf = lp.connect( f_pyscenic_output, mode='r+', validate=False )
meta = json.loads(zlib.decompress(base64.b64decode( lf.attrs.MetaData )))
#exprMat = pd.DataFrame( lf[:,:], index=lf.ra.Gene, columns=lf.ca.CellID)
auc_mtx = pd.DataFrame( lf.ca.RegulonsAUC, index=lf.ca.CellID)
regulons = lf.ra.Regulons
dr_umap = pd.read_csv( 'scenic_umap.txt', sep='\t', header=0, index_col=0 )
dr_tsne = pd.read_csv( 'scenic_tsne.txt', sep='\t', header=0, index_col=0 )
###

In [ ]:
auc_mtx.columns = auc_mtx.columns.str.replace('\(','_(')
regulons.dtype.names = tuple( [ x.replace("(","_(") for x in regulons.dtype.names ] )
# regulon thresholds
rt = meta['regulonThresholds']
for i,x in enumerate(rt):
    tmp = x.get('regulon').replace("(","_(")
    x.update( {'regulon': tmp} )

In [ ]:
tsneDF = pd.DataFrame(adata.obsm['X_tsne'], columns=['_X', '_Y'])

Embeddings_X = pd.DataFrame( index=lf.ca.CellID )
Embeddings_X = pd.concat( [
        pd.DataFrame(adata.obsm['X_umap'],index=adata.obs.index)[0] ,
        pd.DataFrame(adata.obsm['X_pca'],index=adata.obs.index)[0] ,
        dr_tsne['X'] ,
        dr_umap['X']
    ], sort=False, axis=1, join='outer' )
Embeddings_X.columns = ['1','2','3','4']

Embeddings_Y = pd.DataFrame( index=lf.ca.CellID )
Embeddings_Y = pd.concat( [
        pd.DataFrame(adata.obsm['X_umap'],index=adata.obs.index)[1] ,
        pd.DataFrame(adata.obsm['X_pca'],index=adata.obs.index)[1] ,
        dr_tsne['Y'] ,
        dr_umap['Y']
    ], sort=False, axis=1, join='outer' )
Embeddings_Y.columns = ['1','2','3','4']